In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

with open("/kaggle/input/pride-prejudice-subtitles-and-text/PP.txt", "r", encoding="utf8") as f:
    data = f.read()

text = data.replace('\n', ' ').replace('\r', ' ').replace('\ufeff', ' ').replace('“', '').replace('”', '')
text = ' '.join(text.split())

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

with open('token.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle)

vocab_size = len(tokenizer.word_index) + 1
print(f"Taille du vocabulaire : {vocab_size}")

text = text[:100000]


Taille du vocabulaire : 6745


In [2]:
sequence_data = tokenizer.texts_to_sequences([text])[0]
sequences = []
for i in range(5, len(sequence_data)):
    words = sequence_data[i-5:i+1]
    sequences.append(words)

sequences = np.array(sequences)




In [3]:
# Diviser les données en entrée et sortie
X = sequences[:, :-1]  
y = sequences[:, -1]  
y = to_categorical(y, num_classes=vocab_size)





In [4]:
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Embedding(vocab_size, 50))
model.add(Dropout(0.3))
model.add(Bidirectional(GRU(150)))
model.add(Dropout(0.3))
model.add(Dense(vocab_size, activation='softmax'))
 

In [5]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)



Epoch 1/100
568/568 ━━━━━━━━━━━━━━━━━━━━ 13s 12ms/step - accuracy: 0.0324 - loss: 7.0490
Epoch 2/100
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.0476 - loss: 6.0523
Epoch 3/100
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.0713 - loss: 5.7427
Epoch 4/100
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.0899 - loss: 5.4833
Epoch 5/100
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.1050 - loss: 5.2680
Epoch 6/100
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.1182 - loss: 5.0039
Epoch 7/100
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.1332 - loss: 4.7684
Epoch 8/100
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 13ms/step - accuracy: 0.1462 - loss: 4.5139
Epoch 9/100
568/568 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.1604 - loss: 4.2406
Epoch 10/100
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.1844 - loss: 4.0073
Epoch 11/100
568/568 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.2106 - loss: 3.7637
Epoch 12/100
568/568 ━━━━━━━━

In [6]:
input_text = "you have a bad financial"
predict_next_words = 45


for _ in range(predict_next_words):
    input_sequence = tokenizer.texts_to_sequences([input_text])[0]
    
    input_sequence = input_sequence[-5:]
    
    input_sequence = np.array(input_sequence).reshape(1, -1)
    
    predicted_prob = model.predict(input_sequence, verbose=0)
    predicted_index = np.argmax(predicted_prob, axis=-1)
    
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            output_word = word
            break
    
    if output_word == "":
        print("Aucun mot prédit trouvé.")
        break
    
    input_text += " " + output_word

print(input_text)

you have a bad financial cold and how excessively they disliked being ill themselves and then thought no more of the matter and their indifference towards jane when not immediately before them restored elizabeth to the enjoyment of all her former dislike their brother indeed was the only one of
